In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [8]:
### changes in physics - seasonal avg
sst_delta = 0.01 #change in SST, deg/yr?
mld_delta = 1 #change in MLD, m/yr
ot_delta = 0.02 # change in OT, Sv/yr (SV = 1e6 m3/s)

#### gradients in BGC
grad_dDICdz = 0.28 # (umol/L/m) average between 0-100m depth and 100-200m
grad_DIC_surfdeep = 120 #umol/L between surface and 1000m depth
grad_dFedz = 0.0007 #(nmol/L/m) (nmol/L/m)
grad_dFev = 0.4 # deep minus surface ocean (nmol/L) between surface and 1000m depth (based on Tagliabue 2012 database)

### mean ecosystem parameters
mu_L = 1 # 0-1 depending on month  (Light limitation; no units)
dmudFe = 0.59 # (1/d/(nmolFe/L)) see worksheet Fer parameters "C-estimations-v2"
diatconc = 1 # mean diatom concentration (umol/L)

### carbonate system variables
mn_kg = 0.065 # (mol/m2/yr/ppm) xx-TJ I don't know what this is
mn_SST = 2 #(oC)
mn_Revfact = 14 # Definition: R=(dCO2/CO2)/(dDIC/DIC)
mn_DIC = 2120 # (umol/L)
mn_TA = 2290 # (ueq/L)
mn_pCO2 = 380 # (ppm)

# conversions/ geography
PgC_mol = 1.2e-14 # pgC/mol
SO_area = 5.94e13 # assuming 1/6 of the global ocean south of 50S, calculate this from mesh mask


In [9]:
## plot 

In [10]:
def effect_SST(mn_kg,mn_pCO2,sst_delta,PgC_mol,SO_area):
    
    dfco2 = mn_kg*0.04*mn_pCO2*sst_delta #dfCO2 = kg(4%-pCO2/degree x dSST)
    dfco2_pgCyr_SST = dfco2 * PgC_mol * SO_area #dfCO2 in PgC/yr

    return dfco2_pgCyr_SST

def effect_MLD(mld_delta, grad_dDICdz, mn_Revfact, mn_pCO2, mn_DIC, mn_kg, PgC_mol, SO_area,\
                grad_dFedz, dmudFe, mu_L, diatconc):
    #dic
    dDIC = mld_delta*grad_dDICdz #dDIC = dMLD x dDIC/dz, umol/L
    dpCO2 = mn_Revfact * mn_pCO2 / mn_DIC * dDIC #dpCO2 = R x pCO2/DIC x dDIC, ppm
    dfCO2_DIC = mn_kg * dpCO2 #dfCO2_DIC = kg x dpCO2
    dfco2_pgCyr_MLD_DIC = dfCO2_DIC * PgC_mol * SO_area 

    ##ta
    dfcCO2_TA = (-1/3)*dfCO2_DIC
    dfco2_pgCyr_MLD_TA = dfcCO2_TA * PgC_mol * SO_area 

    #iron (ecosystem)
    dFe = mld_delta *  grad_dFedz #dFe = dMLD x dFe/dz 
    #print(dFe)
    dDIC_fe = -1 * dmudFe * dFe * mu_L * diatconc * 365.25 #dDIC = -dmu/dFe x dFe x mu_L x DIA x dtime  
    #print(dDIC_fe)
    dpCO2_Fe = mn_Revfact * mn_pCO2 / mn_DIC * dDIC_fe  #dpCO2 = R x pCO2/DIC x dDIC
    #print(dpCO2_Fe)
    dfCO2_Fe = mn_kg * dpCO2_Fe
    #print(dfCO2_Fe)
    dfco2_pgCyr_MLD_Fe = dfCO2_Fe * PgC_mol * SO_area
    #print(dfco2_pgCyr_MLD_Fe)

    return dfco2_pgCyr_MLD_DIC, dfco2_pgCyr_MLD_TA, dfco2_pgCyr_MLD_Fe

def effect_OT(ot_delta,grad_DIC_surfdeep, dmudFe, grad_dFev, mu_L, diatconc):

    dfco2_pgCyr_OT_DIC = ot_delta * 1e6 * grad_DIC_surfdeep * 1000*31560000*0.000001*0.000000000000012 #dfCO2_DIC = dOverturning x (dDICv)
    dfco2_pgCyr_OT_TA = -(1/3)*dfco2_pgCyr_OT_DIC #dfCO2_TA = -1/3 x dfCO2_DIC
    #dDICv = -dmu/dFe x dFev x mu_L x DIA x dtime 
    dDICv = -dmudFe* grad_dFev * mu_L * diatconc * 365.25 #dDICv = -dmu/dFe x dFev x mu_L x DIA x dtime 
    dfco2_pgCyr_OT_Fe = dDICv * ot_delta * 1e6 * 1000*31560000*0.000001*0.000000000000012 # dfCO2_DIC = dOverturning x (dDIC)

    return dfco2_pgCyr_OT_DIC, dfco2_pgCyr_OT_TA, dfco2_pgCyr_OT_Fe


def give_me_summary(mn_kg,mn_pCO2,sst_delta,PgC_mol,SO_area,\
                    mld_delta, grad_dDICdz, mn_Revfact, mn_DIC, grad_dFedz, dmudFe, mu_L, diatconc,\
                    ot_delta,grad_DIC_surfdeep, grad_dFev, verbose = False): 

    dfco2_pgCyr_SST  = effect_SST(mn_kg,mn_pCO2,sst_delta,PgC_mol,SO_area)

    dfco2_pgCyr_MLD_DIC, dfco2_pgCyr_MLD_TA, dfco2_pgCyr_MLD_Fe = \
        effect_MLD(mld_delta, grad_dDICdz, mn_Revfact, mn_pCO2, mn_DIC, mn_kg, PgC_mol, SO_area, grad_dFedz, dmudFe, mu_L, diatconc)

    dfco2_pgCyr_OT_DIC, dfco2_pgCyr_OT_TA, dfco2_pgCyr_OT_Fe = \
        effect_OT(ot_delta,grad_DIC_surfdeep, dmudFe, grad_dFev, mu_L, diatconc)
    
    CO2_SST = dfco2_pgCyr_SST
    CO2_MLD_phys_chem = dfco2_pgCyr_MLD_TA + dfco2_pgCyr_MLD_DIC
    CO2_OT_phys_chem = dfco2_pgCyr_OT_TA + dfco2_pgCyr_OT_DIC
    CO2_OT_MLD_bio = dfco2_pgCyr_OT_Fe + dfco2_pgCyr_MLD_Fe
    CO2_tot = CO2_SST + CO2_MLD_phys_chem + CO2_OT_phys_chem + CO2_OT_MLD_bio
    if verbose:
        
        print('individual effects')
        print(f'effect SST pgC/yr {dfco2_pgCyr_SST}')
        print(f'effect MLD (TA, DIC, Fe) pgC/yr {dfco2_pgCyr_MLD_DIC}, {dfco2_pgCyr_MLD_TA}, {dfco2_pgCyr_MLD_Fe}')
        print(f'effect OT (TA, DIC, Fe) pgC/yr {dfco2_pgCyr_OT_DIC}, {dfco2_pgCyr_OT_TA}, {dfco2_pgCyr_OT_Fe}')

        print('')
        print('--summary (in PgC/yr)')
        print(f'Change in CO2 flux due to SST change {CO2_SST}')
        print(f'Change in CO2 flux due to MLD change (combined physico-chemical effects) {CO2_MLD_phys_chem}')
        print(f'Change in CO2 flux due to Overturning change (combined physico-chemical effects) {CO2_OT_phys_chem}')
        print(f'Change in CO2 flux due to MLD+Overturning change (ecosystem effects) {CO2_OT_MLD_bio}')
        print(f'Total change in CO2 flux {CO2_tot}')

        return CO2_SST, CO2_MLD_phys_chem, CO2_OT_phys_chem, CO2_OT_MLD_bio, CO2_tot

#         dfco2_pgCyr_SST = effect_SST(mn_kg,mn_pCO2,sst_delta,PgC_mol,SO_area)
#         dfco2_pgCyr_MLD_DIC, dfco2_pgCyr_MLD_TA, dfco2_pgCyr_MLD_Fe = effect_MLD(mld_delta, grad_dDICdz, mn_Revfact, mn_pCO2, mn_DIC, mn_kg, PgC_mol, SO_area)


# Check duplication of CLQ's math in dummy problem 

In [11]:


dfco2_pgCyr_SST  = effect_SST(mn_kg,mn_pCO2,sst_delta,PgC_mol,SO_area)
print(f'effect SST pgC/yr {dfco2_pgCyr_SST}')
dfco2_pgCyr_MLD_DIC, dfco2_pgCyr_MLD_TA, dfco2_pgCyr_MLD_Fe = \
    effect_MLD(mld_delta, grad_dDICdz, mn_Revfact, mn_pCO2, mn_DIC, mn_kg, PgC_mol, SO_area, grad_dFedz, dmudFe, mu_L, diatconc)
print(f'effect MLD (TA, DIC, Fe) pgC/yr {dfco2_pgCyr_MLD_DIC}, {dfco2_pgCyr_MLD_TA}, {dfco2_pgCyr_MLD_Fe}')
dfco2_pgCyr_OT_DIC, dfco2_pgCyr_OT_TA, dfco2_pgCyr_OT_Fe = \
    effect_OT(ot_delta,grad_DIC_surfdeep, dmudFe, grad_dFev, mu_L, diatconc)
print(f'effect OT (TA, DIC, Fe) pgC/yr {dfco2_pgCyr_OT_DIC}, {dfco2_pgCyr_OT_TA}, {dfco2_pgCyr_OT_Fe}')

# dfco2_pgCyr_OT_DIC, dfco2_pgCyr_OT_TA, dfco2_pgCyr_OT_Fe = effect_OT(ot_delta,grad_DIC_surfdeep, dmudFe, grad_dFev, mu_L, diatconc)
# print(f'effect OT (TA, DIC, Fe) pgC/yr {dfco2_pgCyr_OT_DIC}, {dfco2_pgCyr_OT_TA}, {dfco2_pgCyr_OT_Fe}')

CO2_SST, CO2_MLD_phys_chem, CO2_OT_phys_chem, CO2_OT_MLD_bio, CO2_tot = give_me_summary(mn_kg,mn_pCO2,sst_delta,PgC_mol,SO_area,\
                    mld_delta, grad_dDICdz, mn_Revfact, mn_DIC, grad_dFedz, dmudFe, mu_L, diatconc,\
                    ot_delta,grad_DIC_surfdeep, grad_dFev, verbose = True)


effect SST pgC/yr 0.007042464000000002
effect MLD (TA, DIC, Fe) pgC/yr 0.03255478641509434, -0.010851595471698113, -0.01753868771371698
effect OT (TA, DIC, Fe) pgC/yr 0.0009089279999999999, -0.00030297599999999994, -0.0006529057055999999
individual effects
effect SST pgC/yr 0.007042464000000002
effect MLD (TA, DIC, Fe) pgC/yr 0.03255478641509434, -0.010851595471698113, -0.01753868771371698
effect OT (TA, DIC, Fe) pgC/yr 0.0009089279999999999, -0.00030297599999999994, -0.0006529057055999999

--summary (in PgC/yr)
Change in CO2 flux due to SST change 0.007042464000000002
Change in CO2 flux due to MLD change (combined physico-chemical effects) 0.02170319094339623
Change in CO2 flux due to Overturning change (combined physico-chemical effects) 0.000605952
Change in CO2 flux due to MLD+Overturning change (ecosystem effects) -0.01819159341931698
Total change in CO2 flux 0.011160013524079252


In [24]:
sst_delta = 0.01 #change in SST, deg/yr?
mld_delta = 1 #change in MLD, m/yr
ot_delta = 0.02 # change in OT, Sv/yr (SV = 1e6 m3/s)

#### gradients in BGC
grad_dDICdz = 0.28 # (umol/L/m) average between 0-100m depth and 100-200m
grad_DIC_surfdeep = 120 #umol/L between surface and 1000m depth
grad_dFedz = 0.0007 #(nmol/L/m) (nmol/L/m)
grad_dFev = 0.4 # deep minus surface ocean (nmol/L) between surface and 1000m depth (based on Tagliabue 2012 database)

### mean ecosystem parameters
mu_L = 1 # 0-1 depending on month  (Light limitation; no units)
dmudFe = 0.59 # (1/d/(nmolFe/L)) see worksheet Fer parameters "C-estimations-v2"
diatconc = 1 # mean diatom concentration (umol/L)

### carbonate system variables
mn_kg = 0.065 # (mol/m2/yr/ppm) xx-TJ I don't know what this is
mn_SST = 2 #(oC)
mn_Revfact = 14 # Definition: R=(dCO2/CO2)/(dDIC/DIC)
mn_DIC = 2120 # (umol/L)
mn_TA = 2290 # (ueq/L)
mn_pCO2 = 380 # (ppm)


yr = 1950
nam = f'test-{yr}.nc'
times = pd.date_range(f"{yr}/01/01",f"{yr}/12/11",freq='MS')#,closed='left')
print(times)
data_vars = {
    'sst_delta_ozone': (['time_counter', 'quantity'], np.zeros([12,4]),
             {'units': ''}),                       
   'mld_delta_ozone': (['time_counter', 'quantity'], np.zeros([12,4]),
            {'units': ''}),
    'ot_delta_ozone': (['time_counter', 'quantity'], np.zeros([12,4]),
            {'units': ''}),

    'sst_delta_ghg': (['time_counter', 'quantity'], np.zeros([12,4]),
            {'units': ''}),
    'mld_delta_ghg': (['time_counter', 'quantity'], np.zeros([12,4]),
            {'units': ''}),
    'ot_delta_ghg': (['time_counter', 'quantity'], np.zeros([12,4]),
            {'units': ''}),

    'sst_delta_tot': (['time_counter', 'quantity'], np.zeros([12,4]),
            {'units': ''}),
    'mld_delta_tot': (['time_counter', 'quantity'], np.zeros([12,4]),
            {'units': ''}),
    'ot_delta_tot': (['time_counter', 'quantity'], np.zeros([12,4]),
            {'units': ''}),

        ### bgc gradients
    'grad_dDICdz': (['time_counter', 'quantity'], np.zeros([12,4]),
            {'units': ''}),
    'grad_DIC_surfdeep': (['time_counter', 'quantity'], np.zeros([12,4]),
            {'units': ''}),
    'grad_dFedz': (['time_counter', 'quantity'], np.zeros([12,4]),
            {'units': ''}),
    'grad_dFev': (['time_counter', 'quantity'], np.zeros([12,4]),
            {'units': ''}),

        ### bgc gradients
    'mu_L': (['time_counter', 'quantity'], np.zeros([12,4]),
            {'units': ''}),
    'dmudFe': (['time_counter', 'quantity'], np.zeros([12,4]),
            {'units': ''}),
    'diatconc': (['time_counter', 'quantity'], np.zeros([12,4]),
            {'units': ''}),

        ### carbonate system variables
    'mn_kg': (['time_counter', 'quantity'], np.zeros([12,4]),
            {'units': ''}),
    'mn_SST': (['time_counter', 'quantity'], np.zeros([12,4]),
            {'units': ''}),
    'mn_Revfact': (['time_counter', 'quantity'], np.zeros([12,4]),
            {'units': ''}),
    'mn_DIC': (['time_counter', 'quantity'], np.zeros([12,4]),
            {'units': ''}),
    'mn_TA': (['time_counter', 'quantity'], np.zeros([12,4]),
            {'units': ''}),
    'mn_pCO2': (['time_counter', 'quantity'], np.zeros([12,4]),
            {'units': ''}),




}

coords = {'time_counter': (['time_counter'], times),
          'quantity': (['mean', 'stdev', 'max', 'min']),
        #   'stdev': (['stdev'], 'stdev'),
        #   'max': (['max'], 'max'),
        #   'min': (['min'], 'min'),


}
# define global attributes
attrs = {'made in':'BOE-SO-carbon/sketch.ipynb',
'desc': ''
}
ds = xr.Dataset(data_vars=data_vars,
coords=coords,
attrs=attrs)
# ds.to_netcdf(savenam)

DatetimeIndex(['1950-01-01', '1950-02-01', '1950-03-01', '1950-04-01',
               '1950-05-01', '1950-06-01', '1950-07-01', '1950-08-01',
               '1950-09-01', '1950-10-01', '1950-11-01', '1950-12-01'],
              dtype='datetime64[ns]', freq='MS')


ValueError: Could not convert tuple of form (dims, data[, attrs, encoding]): (['time_counter', 'mean', 'stdev', 'max', 'min'], array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]]), {'units': ''}) to Variable.

In [23]:
ds

<xarray.Dataset>
Dimensions:          (time_counter: 12, quantity: 4)
Coordinates:
  * time_counter     (time_counter) datetime64[ns] 1950-01-01 ... 1950-12-01
  * quantity         (quantity) <U1 'a' 'b' 'c' 'd'
Data variables:
    sst_delta_ozone  (time_counter, quantity) float64 0.0 0.0 0.0 ... 0.0 0.0
Attributes:
    made in:  BOE-SO-carbon/sketch.ipynb
    desc: